In [1]:
import nibabel as nib
import os, sys
import numpy as np
from tqdm import tqdm

## Functions

In [2]:
def saveImage(Image , Affine , Header , outDirectory):
    """ Inputs:  Image , Affine , Header , outDirectory """
    out = nib.Nifti1Image((Image).astype('float32'),Affine)
    out.get_header = Header
    nib.save(out , outDirectory)

    
def fuse_nuclei(Directory, nuclei_tag='_PProcessed'):
    """ Saving all of the predicted nuclei into one nifti image

    Args:
        Directory (str): The path to all predicted nuclei
        mode (str, optional): Optional tag that can be added to the predicted nuclei names. Defaults to '_PProcessed'.
    """

    Directory = os.path.abspath(Directory) + '/'

    Nuclei = {
        1: '1-THALAMUS',
        2: '2-AV',
        4: '4-VA',
        5: '5-VLa',
        6: '6-VLP',
        7: '7-VPL',
        8: '8-Pul',
        9: '9-LGN',
        10: '10-MGN',
        11: '11-CM',
        12: '12-MD-Pf',
        13: '13-Hb',
        14: '14-MTT'}
            
    mask = []

    # Looping through all nuclei
    for cnt, name in zip( Nuclei.keys() , Nuclei.values() ):

        dirr = Directory + '/' + name + nuclei_tag + '.nii.gz'

        if not os.path.isfile(dirr): continue

        if cnt == 1:

            # loading thalamus mask for the purpose of using its affine matrices & header
            assert os.path.isfile(dirr), 'Thalamus mask does not exist'
            thalamus_mask = nib.load(dirr)

        else:

            # saving the nuclei into one mask
            msk = nib.load(dirr).get_data() > 0.5
            
            if mask == []:
                # saving the first nucleus (2-AV)
                mask = cnt * msk

            else:
                # saving the remaining nuclei, while randomly assigning a label from the labels 
                # that exist in the overlapping area
                mask_temp = mask.copy()
                mask_temp[msk == 0] = 0
                x = np.where(mask_temp > 0)
                if x[0].shape[0] > 0:
                    fg = np.random.randn(x[0].shape[0])
                    fg1, fg2 = fg >= 0, fg < 0
                    mask[x[0][fg1], x[1][fg1], x[2][fg1]] = 0
                    msk[x[0][fg2], x[1][fg2], x[2][fg2]] = 0

                mask += cnt * msk

    # Saving the final multi-label segmentaion mask as a nifti image
    saveImage(mask, thalamus_mask.affine, thalamus_mask.header, Directory + '/AllLabels.nii.gz')


## Test on One Example

In [5]:
Directory = '/array/hdd/msmajdi/data/preprocessed/data_experiment/test/case_9_UA_Siemens_WMn/Label'

fuse_nuclei(Directory=Directory, nuclei_tag='_PProcessed')

# nib.viewers.OrthoSlicer3D(merged_Mask).show()

/array/ssd/msmajdi/anaconda3/envs/new-env-fb/lib/python3.6/site-packages/ipykernel_launcher.py:53: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


## Running on All Cases

In [7]:
Directory = '/array/hdd/msmajdi/data/preprocessed/data_experiment/test/'

subjects = next( os.walk(Directory) )[1]
for cnt, subj in enumerate(subjects):
    print(str(cnt)+'/'+str(len(subjects)-1), subj)
    # merging_Nuclei(Directory=Directory_Predictions + vimp , nuclei_tag='')    # for Predictions
    fuse_nuclei(Directory=Directory + subj + '/Label', nuclei_tag='_PProcessed')  # for Manual Labels


0/22 case_17_UCLA_Siemens_WMn
/array/ssd/msmajdi/anaconda3/envs/new-env-fb/lib/python3.6/site-packages/ipykernel_launcher.py:53: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
1/22 case_19_UCLA_Siemens_WMn
2/22 case_20_UCLA_Siemens_WMn
3/22 case_21_UCLA_Siemens_WMn
4/22 case_7_UA_Siemens_WMn
5/22 case_884_06272013_TS_GE_WMn
6/22 case_8_UA_Siemens_WMn
7/22 case_901_07052013_AS_GE_WMn
8/22 case_915_07112013_LC_GE_WMn
9/22 case_943_07242013_PA_GE_WMn
10/22 case_964_08092013_TG_GE_WMn
11/22 case_9_UA_Siemens_WMn
12/22 case_ctrl_920_07122013_SW_GE_WMn
13/22 case_ctrl_921_07122013_MP_GE_WMn
14/22 case_ctrl_925_07152013_LS_GE_WMn
15/22 case_F_3T_ET_GE_WMn
16/22 case_F_7T_ET_GE_WMn
17/22 case_H_3T_ET_GE_WMn
18/22 case_H_7T_ET_GE_WMn
19/22 case_I_3T_ET_GE_WMn
20/22 case_I_7T_ET_GE_WMn
21/22 case_L_3T_ET_GE_WMn
22/22 case_L_7T_ET_GE_WMn
